In [ ]:
#| export
from sqlite_minutils.utils import ( chunks, hash_record, sqlite3, OperationalError, suggest_column_types, types_for_column_types, column_affinity, find_spatialite,)
import binascii
from collections import namedtuple
from collections.abc import Mapping
import contextlib, datetime, decimal, inspect, itertools, json, os, pathlib, re, secrets, textwrap
from typing import ( cast, Any, Callable, Dict, Generator, Iterable, Union, Optional, List, Tuple,)
from functools import cache
import uuid

In [ ]:
from fastcore.utils import *

In [ ]:
#| export
try: from sqlite_dump import iterdump
except ImportError: iterdump = None

In [ ]:
#| export
__all__ = ['Database', 'Queryable', 'Table', 'View']

In [ ]:
#| export
class Database:
    """
    Wrapper for a SQLite database connection that adds a variety of useful utility methods.

    To create an instance::

        # create data.db file, or open existing:
        db = Database("data.db")
        # Create an in-memory database:
        db_mem = Database(memory=True)
    """

    _counts_table_name = "_counts"
    use_counts_table = False

    def __init__(
        self,
        filename_or_conn: Optional[Union[str, pathlib.Path, sqlite3.Connection]] = None, # String path to a file, or a `pathlib.Path` object, or a `sqlite3` connection
        memory: bool = False,              # set to `True` to create an in-memory database
        memory_name: Optional[str] = None, # reates a named in-memory database that can be shared across multiple connections
        recreate: bool = False,            # set to `True` to delete and recreate a file database (**dangerous**)
        recursive_triggers: bool = True,   # defaults to `True`, which sets `PRAGMA recursive_triggers=on;` - set to `False` to avoid setting this pragma
        tracer: Optional[Callable] = None, # set a tracer function (`print` works for this) which will be called with `sql, parameters` every time a SQL query is executed
        use_counts_table: bool = False,    # set to `True` to use a cached counts table, if available. See :ref:`python_api_cached_table_counts`
        strict: bool = False,              # Apply STRICT mode to all created tables (unless overridden)
    ):
        assert (filename_or_conn is not None and (not memory and not memory_name)) or (
            filename_or_conn is None and (memory or memory_name)
        ), "Either specify a filename_or_conn or pass memory=True"
        if memory_name:
            uri = "file:{}?mode=memory&cache=shared".format(memory_name)
            self.conn = sqlite3.connect(
                uri,
                uri=True,
                check_same_thread=False,
                isolation_level=None
            )
        elif memory or filename_or_conn == ":memory:":
            self.conn = sqlite3.connect(":memory:", isolation_level=None)
        elif isinstance(filename_or_conn, (str, pathlib.Path)):
            if recreate and os.path.exists(filename_or_conn):
                try:
                    os.remove(filename_or_conn)
                except OSError:
                    # Avoid mypy and __repr__ errors, see:
                    # https://github.com/simonw/sqlite-utils/issues/503
                    self.conn = sqlite3.connect(":memory:", isolation_level=None)
                    raise
            self.conn = sqlite3.connect(str(filename_or_conn), check_same_thread=False, isolation_level=None)
        else:
            assert not recreate, "recreate cannot be used with connections, only paths"
            self.conn = filename_or_conn
        if not hasattr(self.conn, '__enter__'):
            self.conn.__enter__ = __conn_enter__
            self.conn.__exit__ = __conn_exit__
        self._tracer = tracer
        if recursive_triggers:
            self.execute("PRAGMA recursive_triggers=on;")
        self._registered_functions: set = set()
        self.use_counts_table = use_counts_table
        self.strict = strict

In [ ]:
#| export
@patch
def execute(
    self:Database, sql: str, parameters: Optional[Union[Iterable, dict]] = None
) -> sqlite3.Cursor:
    """
    Execute SQL query and return a ``sqlite3.Cursor``.

    :param sql: SQL query to execute
    :param parameters: Parameters to use in that query - an iterable for ``where id = ?``
      parameters, or a dictionary for ``where id = :id``
    """
    if self._tracer:
        self._tracer(sql, parameters)
    if parameters is not None:
        return self.conn.execute(sql, tuple(parameters))
    else:
        return self.conn.execute(sql)


In [ ]:

def close(self):
    "Close the SQLite connection, and the underlying database file"
    self.conn.close()

In [ ]:
#| export
@patch
def get_last_rowid(self:Database) -> int|None:
    res = next(self.execute('SELECT last_insert_rowid()'), None)
    if res is None: return None
    return int(res[0])

## Tests

In [ ]:
db = Database('users.db')
db.get_last_rowid()

0